## Snowflake Connector Sample Notebook

In [1]:
import yaml
config = yaml.load(open('./config.yaml', 'rb'), Loader=yaml.FullLoader)
from sfutils import *
sf_connect = SnowflakeSetup(config['snowflake'])
ctx = sf_connect.ctx

### Example from SnowFlake (Will Fail)
source: https://docs.snowflake.com/en/user-guide/python-connector-api.html#write_pandas

In [7]:
# Example from SnowFlake (Will Fail)
from snowflake.connector.pandas_tools import write_pandas
import pandas
df = pandas.DataFrame([('Mark', 10), ('Luke', 20)], columns=['name', 'balance'])

# Mask the error with try/except
try:
    success, nchunks, nrows, _ = write_pandas(ctx, df, 'acustomer')
except:
    print('\033[91m WARNING: Snowflake Error SQL compliation Error Table '"acustomer"' does not exist')

### Using Snowflake Util will run without errors

In [10]:
# Using Snowflake Util will run without errors
try:
    sf_connect.write_to_sf(df, 'acustomers', 'replace')
    print('\033[92m SUCCESS')
except:
    raise

 SUCCESS


## Multi SQL Statements Comparison

In [18]:
conn = sf_connect.ctx
sql_string = """
    SELECT * FROM [ENTER YOUR TABLE NAME] --7.4 million
    ;
    SELECT * FROM [ENTER YOUR TABLE NAME] --19 rows
    ;
"""

In [21]:
%%timeit -n 7 -r 10
# built in snowflake connector method
conn.execute_string(sql_string)

214 ms ± 29.9 ms per loop (mean ± std. dev. of 10 runs, 7 loops each)


In [22]:
%%timeit -n 7 -r 10
# User defined utility function
sf_connect.run_sf_multi_sql(sql_string)

211 ms ± 17.8 ms per loop (mean ± std. dev. of 10 runs, 7 loops each)
